# Модуль 3

## Загружаем датасет из предыдущего модуля и немножко его преобрабатываем

В этой сессии снова сделаем разбиение на обучающую и тестовую выборки для более простого импорта одного датасета, вместо 4

In [7]:
#Импортируем библиотеки
import pandas as pd #библиотека для работы с датасетом
import warnings #игнорирование ошибок
warnings.filterwarnings("ignore")

In [8]:
#Загружаем обработанный датасет в предыдущей сессии 
df = pd.read_csv('C3_M2.csv')
df = df.drop('Unnamed: 0', axis=1)

In [9]:
df #Проверяем датасет на корректность импорта

,light,region,address,category,severity,injured_count,parent_region,road_conditions,participants_count
0,4,150,110982,16,0,4,0,3912,5
1,4,1429,75284,16,2,3,0,3912,4
2,4,1830,75232,12,2,1,0,3912,2
3,2,815,26278,9,1,0,0,417,2
4,0,150,112217,9,2,1,0,2738,2
...,...,...,...,...,...,...,...,...,...
735091,0,252,390265,9,2,1,84,417,2
735092,4,252,91218,11,1,0,84,4365,3
735093,2,252,12981,2,1,0,84,3912,2
735094,4,252,91221,16,0,1,84,4364,3


## 3.1 Визуализация зависимостей данных

Для классификации надо понять, зависят ли атрибуты друг от друга и, если да, то какие и как. Для этого использует тепловую карту и матрицу.

### Тепловая карта

In [ ]:
# Импорт библиотеки
import seaborn as sns
# Отображение тепловой карты
sns.heatmap(df.corr());

Интерпретировав результат можно сказать, что адреса не зависят от региона, поэтому эти данные удалять не стоит.

### Матрица

In [ ]:
# Импорт библиотеки
from pandas.plotting import scatter_matrix
# Отображение матрицы
scatter_matrix(df, alpha=0.05, figsize=(10, 10));

С помощью матрицы можно сделать такой же вывод, что адреса зависят от региона

## 3.2 Классификация и 3.3 Обучение

Теперь надо понять, какую модель классификации выбрать и обучить на тестовых данных. Для этого я возьму 3 метода классификации: RF, KNN, GBT, посмотрю какой метод справился лучше всех.

Скейлируем масштаб таблицы и нормализуем данные

In [10]:
# Масштабирование таблицы
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.drop('severity', axis=1))
scaled_features = scaler.transform(df.drop('severity', axis=1))
scaled_data = pd.DataFrame(scaled_features, columns = df.drop('severity', axis=1).columns)

# Загружаем библиотеку для разделения на выборки
from sklearn.model_selection import train_test_split

# Разделение в соотношении 66/33 с шагом 42
X_train, X_test, y_train, y_test = train_test_split(scaled_data, df.severity, test_size=0.2, random_state=42)

### Классификация Random Forest

Алгоритм строит ансамбль случайных деревьев, каждое из которых обучается на выборке, полученной из исходной с помощью процедуры изъятия с возвращением.

In [ ]:
# Импортируем библиотеку для обучения
from sklearn import ensemble
import numpy as np

#Обучаем модель
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train, y_train)

# Оценка ошибок
err_train = np.mean(y_train != rf.predict(X_train))
err_test  = np.mean(y_test  != rf.predict(X_test))

In [ ]:
#Вывод процента ошибок
print(err_train, err_test)

Точность 100% и 65%
0.006 0.355

### Классификация K-Neighbors

Для нового объекта алгоритм ищет в обучающей выборке k наиболее близких объекта и относит новый объект к тому классу, которому принадлежит большинство из них.

In [ ]:
# Импортируем библиотеку для обучения
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

#Обучаем модель
knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(X_train, y_train)
y_train_predict = knn.predict(X_train)
y_test_predict = knn.predict(X_test)

# Оценка ошибок
err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)

In [ ]:
#Вывод процента ошибок
print(err_train, err_test)

Точность 66% и 59%
0.338 и 0.419

### Классификация Gradient Boosting

Строит ансамбль деревьев решений. На каждой итерации строится новый классификатор, аппроксимирующий значение градиента функции потерь.

In [ ]:
# Импортируем библиотеку для обучения
from sklearn import ensemble
import numpy as np

#Обучаем модель
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)
gbt.fit(X_train, y_train)

# Оценка ошибок
err_train = np.mean(y_train != gbt.predict(X_train))
err_test  = np.mean(y_test  != gbt.predict(X_test))

In [ ]:
#Вывод процента ошибок
print(err_train, err_test)

Точность 66% и 66%
0.331 и 0.332

Точности моделей недостаточно для достижения желаемого  результата, поэтому путём преобразования набора данных нужно добиться более точной работы выбранных моделей.

## 3.4 Feature Engineering

Для начала сделаем алгоритм, который подберёт лучший вариант параметра n_neighbors с наивысшей точностью. Он позволит нам сразу повысить точность модели

### KNN алгоритм определения параметра

In [ ]:
#Импортируем библиотеку для определения наилучшего параметра n_neighbors
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

error_rates = []
for i in np.arange(1, 101):
    new_model = KNeighborsClassifier(n_neighbors = i)
    new_model.fit(X_train, y_train)
    new_predictions = new_model.predict(X_test)
    error_rates.append(np.mean(new_predictions != y_test))
plt.plot(error_rates)

Алгоритм выдал наилучший результат n_neighbors = 18. Теперь удалим столбцы с большим и малым разбросом данных.

In [11]:
#Создаём новуб таблицу и удаляем данные для Feature Engineering
df_new = df.drop(['region', 'parent_region', 'address'], axis=1) #Удаляем данные о регионе, адресах

In [12]:
#Проверяем созданную таблицу
df_new

,light,category,severity,injured_count,road_conditions,participants_count
0,4,16,0,4,3912,5
1,4,16,2,3,3912,4
2,4,12,2,1,3912,2
3,2,9,1,0,417,2
4,0,9,2,1,2738,2
...,...,...,...,...,...,...
735091,0,9,2,1,417,2
735092,4,11,1,0,4365,3
735093,2,2,1,0,3912,2
735094,4,16,0,1,4364,3


In [13]:
# Масштабирование таблицы
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_new.drop('severity', axis=1))
scaled_features = scaler.transform(df_new.drop('severity', axis=1))
scaled_data = pd.DataFrame(scaled_features, columns = df_new.drop('severity', axis=1).columns)

# Загружаем библиотеку для разделения на выборки
from sklearn.model_selection import train_test_split

# Разделение в соотношении 66/33 с шагом 42
X_train, X_test, y_train, y_test = train_test_split(scaled_data, df.severity, test_size=0.2, random_state=42)

### KNN

Изменённый результат метода KNN с обучением

In [ ]:
#Импортируем библиотеки
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

#Обучаем модель
knn = KNeighborsClassifier(n_neighbors = 18)
knn.fit(X_train, y_train)
y_train_predict = knn.predict(X_train)
y_test_predict = knn.predict(X_test)

# Оценка ошибок
err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test != y_test_predict)

In [ ]:
#Вывод процента ошибок
print(err_train, err_test)

Точность 86% и 91%
0,147 и 0,092

### GBT

GBT метод с обработанным датасетом df_new

In [ ]:
# Импортируем библиотеку для обучения
from sklearn import ensemble
import numpy as np

#Обучаем модель
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)
gbt.fit(X_train, y_train)

# Оценка ошибок
err_train = np.mean(y_train != gbt.predict(X_train))
err_test  = np.mean(y_test  != gbt.predict(X_test))

In [ ]:
#Вывод процента ошибок
print(err_train, err_test)

Точность 88 и 86%
0.222 и 0,245

Из-за слишком большого объёма данных не хватает мощностей для более лучшего анализа моделей классификации, поэтому, к сожалению, можно зафиксировать наилучший показатель на данный момент: **Точность 86% и 91%, 0,147 и 0,092**.

## 3.5 Интерактивная карта

Загрузим данные геопозиции из первой сессии для последующей работы

In [14]:
#Загружаем обработанный датасет в предыдущей сессии 
geo = pd.read_csv('Geoposition.csv')
geo = geo.drop('Unnamed: 0', axis=1)
#Проверяем датасет
geo

,point
0,"{'lat': 53.361674, 'long': 83.770891}"
1,"{'lat': 53.379437, 'long': 83.997624}"
2,"{'lat': 53.852527, 'long': 83.47219}"
3,"{'lat': 52.21802, 'long': 79.190655}"
4,"{'lat': 53.358966, 'long': 83.659204}"
...,...
735091,"{'lat': 57.26, 'long': 39.154}"
735092,"{'lat': 57.270551, 'long': 39.13197}"
735093,"{'lat': 57.149, 'long': 37.794}"
735094,"{'lat': 57.261182, 'long': 39.153482}"


In [15]:
#Разделяем в колонке point долготу и ширину
geo = geo['point'].str.split(',',expand=True)

#Переименовываем столбцы из 0 в lat, из 1 в long
geo.columns=['lat','long']

#Удаляем мешающие символы с lat
geo['lat'] = geo['lat'].str.replace('{','')
geo['lat'] = geo['lat'].str.replace('lat','')
geo['lat'] = geo['lat'].str.replace(':','')
geo['lat'] = geo['lat'].str.replace(' ','')
geo['lat'] = geo['lat'].str.replace("''",'')
#Удаляем мешающие символы с long
geo['long'] = geo['long'].str.replace('}','')
geo['long'] = geo['long'].str.replace('long','')
geo['long'] = geo['long'].str.replace(':','')
geo['long'] = geo['long'].str.replace(' ','')
geo['long'] = geo['long'].str.replace("''",'')
#Удаляем промущенные координаты ширины и долготы
geo = geo[~geo['lat'].isin(['None'])]
geo = geo[~geo['long'].isin(['None'])]
#Конвертируем в вещественные числа
geo['lat'] = geo['lat'].astype(float)
geo['long'] = geo['long'].astype(float)

In [16]:
#Проверяем таблицу после обработки
geo

,lat,long
0,53.361674,83.770891
1,53.379437,83.997624
2,53.852527,83.472190
3,52.218020,79.190655
4,53.358966,83.659204
...,...,...
735091,57.260000,39.154000
735092,57.270551,39.131970
735093,57.149000,37.794000
735094,57.261182,39.153482


Выглядит красиво, теперь с помощью библиотеки folium сделаем карту

In [ ]:
#Импортируем библиотеку для создания карт
import folium
#Создаём карту с появлением по средней наших координат, начальный зум 14 с возможностью интерактивного взаимодействия
map = folium.Map(location=[geo.lat.mean(), geo.long.mean()], zoom_start=14, control_scale=True)

In [ ]:
map

Теперь добавим маркеры на нужные нам геопозиции и покрасим их в разные цвета

In [ ]:
#Создаём карту с появлением по средней наших координат, начальный зум 14 с возможностью интерактивного взаимодействия
map = folium.Map(location=[geo.lat.mean(), geo.long.mean()], zoom_start=14, control_scale=True)
#Добавляем маркеры, цвет, позицию на карту
for index, location_info in geo.iterrows():
    folium.Marker([location_info["lat"], location_info["long"]]).add_to(map)

Сохраняем карту в HTML страницу

In [ ]:
#Сохранение интерактивной карты
map.save('map.html')

## 3.6 Отчёт

Итог сессии: Слишком большой объём данных был получен для выполнения задания, аппарат, на котором производилась работа не смог быстро и корректно отобразить желаемый результат обучения моделей и интерактивной карты

In [17]:
#Сохранение в CSV
df.to_csv('C3_M3.csv')
df_new.to_csv('C3_M3_new.csv')
geo.to_csv('Geoposition.csv')

In [ ]:
#Сохранение HTML
!jupyter nbconvert C3_M3.ipynb --to html